In [18]:
import torch
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

import regularization as R
from modules import NMU, NAU, LeibnizModule
from samplers import *
from datasets import MatrixDeterminantDataset, BatchDataLoader

torch.cuda.set_device(1)

In [19]:
# def reg(W):
#     return W**2 * (1 - W)**2

# def regl(W):
#     W_abs = np.abs(W)
#     return np.minimum(np.abs(W), np.abs(1 - W))

# Ws = np.linspace(0, 1, num=10002)
# plt.plot(Ws, regl(Ws), label="linear")
# plt.plot(Ws, reg(Ws)*6, label="squared")
# plt.legend()
# plt.yticks([])

In [20]:
def train_until_convergence(
    model,
    train_loader,
    dataset_valid_interpolation_data,
    dataset_test_extrapolation_data,
    regualizer_scaling_start=5000,
    max_iter=20000,
    alpha_scale=1.001,
    alpha_start=0.05,
    check_period=250,
    lr=2e-3,
    verbose=False
):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = torch.nn.MSELoss()
    
    def test_model(data):
        with torch.no_grad():
            var, x, t = data
            return criterion(model(x), t) / var
    
    for epoch_i, (var, x_train, t_train) in zip(range(1, max_iter + 1), train_loader):
        optimizer.zero_grad()
        # forward
        y_train = model(x_train)

        if(epoch_i == regualizer_scaling_start):
            r_w_scale = 0.01
        elif(epoch_i > regualizer_scaling_start):
            r_w_scale *= alpha_scale
        else:
            r_w_scale = 0
            
        muls = dict(
            sparsity=r_w_scale,
            n_coeffs=0
        )

        loss_train_regualizer = R.eval_regularizers(model, muls)
        loss_train_criterion = criterion(y_train, t_train) / var
        loss_train = loss_train_criterion + loss_train_regualizer
        
        if(epoch_i % check_period == 0):
            interpolation_error = test_model(dataset_valid_interpolation_data) 
            extrapolation_error = test_model(dataset_test_extrapolation_data) 
            sparsity_loss = loss_train_regualizer.detach().cpu().numpy()
            if(verbose):
                infos = f"[epoch {epoch_i}] inter: {interpolation_error:.4g}, extra: {extrapolation_error:.4g}"
                if(r_w_scale > 0):
                    infos += f" | reg: {sparsity_loss / r_w_scale:.4g} (scale: {r_w_scale:.4g})"
                print(infos)
            if(r_w_scale > 0):
                if(sparsity_loss / r_w_scale < 1e-3 and interpolation_error < 1e-3 and extrapolation_error < 1e-3):
                    return True

        
        # Optimize model
        if loss_train.requires_grad:
            loss_train.backward()
            optimizer.step()
    return False

def loaders(dataset):
    inter_sampler = uniform(-2, 2)
    train_loader = dataset.dataloader(batch_size=64, samplers=[inter_sampler])
    dataset_valid_interpolation_data = next(iter(dataset.dataloader(batch_size=10000, samplers=[inter_sampler])))
    dataset_test_extrapolation_data = next(iter(dataset.dataloader(batch_size=10000, samplers=[uniform(-4, 4)])))
    return train_loader, dataset_valid_interpolation_data, dataset_test_extrapolation_data

## Full matrices

In [21]:
ms = np.array([
    [1, 4],
    [3, 2]
])

dataset = MatrixDeterminantDataset(ms)
dataset

In [22]:
model = LeibnizModule(4, 5).cuda()
train_until_convergence(
    model,
    *loaders(dataset),
    regualizer_scaling_start=1000,
    verbose=True
)

[epoch 250] inter: 0.0383, extra: 0.06252
[epoch 500] inter: 0.005077, extra: 0.007671
[epoch 750] inter: 0.00133, extra: 0.002065
[epoch 1000] inter: 0.0003611, extra: 0.0004983 | reg: 0.04194 (scale: 0.01)
[epoch 1250] inter: 8.019e-05, extra: 8.351e-05 | reg: 0.03921 (scale: 0.01284)
[epoch 1500] inter: 1.55e-05, extra: 1.091e-05 | reg: 0.0376 (scale: 0.01648)
[epoch 1750] inter: 2.264e-06, extra: 9.299e-07 | reg: 0.03604 (scale: 0.02116)
[epoch 2000] inter: 2.754e-07, extra: 2.084e-07 | reg: 0.03393 (scale: 0.02717)
[epoch 2250] inter: 2.814e-07, extra: 2.857e-07 | reg: 0.03071 (scale: 0.03488)
[epoch 2500] inter: 3.285e-07, extra: 3.348e-07 | reg: 0.02561 (scale: 0.04478)
[epoch 2750] inter: 4.43e-07, extra: 4.542e-07 | reg: 0.01795 (scale: 0.0575)
[epoch 3000] inter: 3.294e-07, extra: 3.33e-07 | reg: 0.009531 (scale: 0.07382)
[epoch 3250] inter: 1.206e-07, extra: 1.197e-07 | reg: 0.005172 (scale: 0.09477)
[epoch 3500] inter: 1.686e-08, extra: 1.677e-08 | reg: 0.003678 (scale: 0.1

True

In [23]:
model.disp_equation(dataset.adapt_alphabet())

$-a_{3}a_{4}+a_{1}a_{2}$

In [24]:
ms = np.array([
    [1, 2, 3],
    [4, 5, 6],
    [7, 8, 9],
])

dataset = MatrixDeterminantDataset(ms)
dataset

In [25]:
model = LeibnizModule(9, 15).cuda()
train_until_convergence(
    model,
    *loaders(dataset),
    regualizer_scaling_start=3000,
    verbose=True
)

[epoch 250] inter: 0.8343, extra: 0.9235
[epoch 500] inter: 0.3254, extra: 0.3667
[epoch 750] inter: 0.1534, extra: 0.204
[epoch 1000] inter: 0.0001967, extra: 0.02129
[epoch 1250] inter: 7.541e-05, extra: 0.006077
[epoch 1500] inter: 3.911e-05, extra: 0.002095
[epoch 1750] inter: 2.471e-05, extra: 0.001264
[epoch 2000] inter: 1.752e-05, extra: 0.0009169
[epoch 2250] inter: 1.327e-05, extra: 0.0008546
[epoch 2500] inter: 1.02e-05, extra: 0.0007724
[epoch 2750] inter: 8.081e-06, extra: 0.0007314
[epoch 3000] inter: 6.323e-06, extra: 0.000523 | reg: 0.05284 (scale: 0.01)
[epoch 3250] inter: 4.988e-06, extra: 0.0005593 | reg: 0.05039 (scale: 0.01284)
[epoch 3500] inter: 4.264e-06, extra: 0.0003683 | reg: 0.04714 (scale: 0.01648)
[epoch 3750] inter: 3.275e-06, extra: 0.0001811 | reg: 0.04283 (scale: 0.02116)
[epoch 4000] inter: 1.418e-06, extra: 8.081e-05 | reg: 0.03669 (scale: 0.02717)
[epoch 4250] inter: 1.53e-07, extra: 9.961e-06 | reg: 0.03081 (scale: 0.03488)
[epoch 4500] inter: 2.199

True

In [26]:
model.disp_equation(dataset.adapt_alphabet())

$-a_{1}a_{6}a_{8}-a_{3}a_{5}a_{7}+a_{1}a_{5}a_{9}+a_{3}a_{4}a_{8}-a_{2}a_{4}a_{9}+a_{2}a_{6}a_{7}$

In [27]:
ms = np.array([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12],
    [13, 14, 15, 16]
])

dataset = MatrixDeterminantDataset(ms)
dataset

In [28]:
model = LeibnizModule(16, 80).cuda()
train_until_convergence(
    model,
    *loaders(dataset),
    regualizer_scaling_start=10000,
    verbose=True
)

[epoch 250] inter: 0.9939, extra: 1.291
[epoch 500] inter: 0.9938, extra: 1.117
[epoch 750] inter: 0.9937, extra: 1.137
[epoch 1000] inter: 0.9198, extra: 1.241
[epoch 1250] inter: 0.8634, extra: 1.321
[epoch 1500] inter: 0.863, extra: 2.919
[epoch 1750] inter: 0.7829, extra: 2.196
[epoch 2000] inter: 0.766, extra: 1.346
[epoch 2250] inter: 0.62, extra: 13.96
[epoch 2500] inter: 0.522, extra: 3.039
[epoch 2750] inter: 0.5188, extra: 3.447
[epoch 3000] inter: 0.4816, extra: 3.081
[epoch 3250] inter: 0.4818, extra: 5.26
[epoch 3500] inter: 0.4594, extra: 2.744
[epoch 3750] inter: 0.4371, extra: 2.559
[epoch 4000] inter: 0.3577, extra: 1.683
[epoch 4250] inter: 0.316, extra: 1.082
[epoch 4500] inter: 0.2836, extra: 1.002
[epoch 4750] inter: 0.2836, extra: 1.21
[epoch 5000] inter: 0.2835, extra: 2.559
[epoch 5250] inter: 0.2836, extra: 2.589
[epoch 5500] inter: 0.2428, extra: 4.639
[epoch 5750] inter: 0.202, extra: 2.118
[epoch 6000] inter: 0.2018, extra: 1.16
[epoch 6250] inter: 0.158, ex

True

In [29]:
model.disp_equation(dataset.adapt_alphabet())

$a_{1}a_{6}a_{11}a_{16}-a_{2}a_{7}a_{12}a_{13}+a_{1}a_{7}a_{12}a_{14}+a_{2}a_{8}a_{11}a_{13}+a_{3}a_{6}a_{12}a_{13}-a_{1}a_{7}a_{10}a_{16}-a_{1}a_{6}a_{12}a_{15}-a_{3}a_{5}a_{12}a_{14}+a_{3}a_{5}a_{10}a_{16}-a_{2}a_{5}a_{11}a_{16}-a_{1}a_{8}a_{11}a_{14}+a_{4}a_{6}a_{9}a_{15}-a_{3}a_{6}a_{9}a_{16}+a_{2}a_{7}a_{9}a_{16}-a_{4}a_{5}a_{10}a_{15}+a_{4}a_{7}a_{10}a_{13}-a_{2}a_{8}a_{9}a_{15}-a_{4}a_{6}a_{11}a_{13}+a_{3}a_{8}a_{9}a_{14}-a_{4}a_{7}a_{9}a_{14}+a_{2}a_{5}a_{12}a_{15}-a_{3}a_{8}a_{10}a_{13}+a_{4}a_{5}a_{11}a_{14}+a_{1}a_{8}a_{10}a_{15}$

## Small structured matrices 

In [30]:
ms = np.array([
    [1, 0, 0, 0],
    [0, 2, 0, 0],
    [0, 0, 3, 0],
    [0, 0, 0, 4]
])

dataset = MatrixDeterminantDataset(ms)
dataset

In [31]:
model = LeibnizModule(4, 3).cuda()
train_until_convergence(
    model,
    *loaders(dataset),
    regualizer_scaling_start=1000,
    verbose=True
)

[epoch 250] inter: 0.01413, extra: 0.003108
[epoch 500] inter: 0.003798, extra: 0.0007319
[epoch 750] inter: 0.001365, extra: 0.0001718
[epoch 1000] inter: 0.0005711, extra: 5.58e-05 | reg: 0.05788 (scale: 0.01)
[epoch 1250] inter: 0.0002311, extra: 2.52e-05 | reg: 0.05475 (scale: 0.01284)
[epoch 1500] inter: 9.101e-05, extra: 9.852e-06 | reg: 0.05277 (scale: 0.01648)
[epoch 1750] inter: 2.939e-05, extra: 3.186e-06 | reg: 0.05134 (scale: 0.02116)
[epoch 2000] inter: 8.545e-06, extra: 1.325e-06 | reg: 0.04991 (scale: 0.02717)
[epoch 2250] inter: 2.161e-06, extra: 2.782e-07 | reg: 0.04793 (scale: 0.03488)
[epoch 2500] inter: 9.18e-07, extra: 2.528e-07 | reg: 0.04472 (scale: 0.04478)
[epoch 2750] inter: 7.294e-07, extra: 1.43e-07 | reg: 0.03933 (scale: 0.0575)
[epoch 3000] inter: 9.766e-07, extra: 2.592e-07 | reg: 0.03068 (scale: 0.07382)
[epoch 3250] inter: 1.41e-06, extra: 4.459e-07 | reg: 0.01885 (scale: 0.09477)
[epoch 3500] inter: 7.341e-07, extra: 2.407e-07 | reg: 0.007068 (scale: 0

True

In [32]:
model.disp_equation(dataset.adapt_alphabet())

$a_{1}a_{2}a_{3}a_{4}$

In [33]:
ms = np.array([
    [1, 0, 0, 2],
    [0, 2, 4, 0],
    [0, 1, 3, 0],
    [3, 0, 0, 4]
])

dataset = MatrixDeterminantDataset(ms, alphabet="abcd", with_multiplicity=True)
dataset

In [34]:
model = LeibnizModule(8, 15).cuda()
train_until_convergence(
    model,
    *loaders(dataset),
    regualizer_scaling_start=5000,
    verbose=True
)

[epoch 250] inter: 0.3204, extra: 5.759
[epoch 500] inter: 0.1778, extra: 1.264
[epoch 750] inter: 0.1683, extra: 0.9172
[epoch 1000] inter: 0.1504, extra: 0.8756
[epoch 1250] inter: 0.08154, extra: 0.7238
[epoch 1500] inter: 0.06285, extra: 1.068
[epoch 1750] inter: 0.05788, extra: 1.095
[epoch 2000] inter: 0.05606, extra: 1.206
[epoch 2250] inter: 0.05476, extra: 1.064
[epoch 2500] inter: 0.05413, extra: 0.9575
[epoch 2750] inter: 0.05208, extra: 0.9202
[epoch 3000] inter: 0.0495, extra: 2.228
[epoch 3250] inter: 0.02492, extra: 3.642
[epoch 3500] inter: 0.008463, extra: 0.7047
[epoch 3750] inter: 0.003773, extra: 0.3846
[epoch 4000] inter: 0.001895, extra: 0.2581
[epoch 4250] inter: 0.001142, extra: 0.2071
[epoch 4500] inter: 0.0007177, extra: 0.1381
[epoch 4750] inter: 0.0004704, extra: 0.1008
[epoch 5000] inter: 0.000338, extra: 0.06413 | reg: 0.02614 (scale: 0.01)
[epoch 5250] inter: 0.0002406, extra: 0.05033 | reg: 0.02469 (scale: 0.01284)
[epoch 5500] inter: 0.0001599, extra: 0

True

In [35]:
model.disp_equation(dataset.adapt_alphabet())

$abdc-adad-bbcc+abcd$

In [41]:
def diagonal_plus_two(n):
    ms = np.zeros((n, n), dtype=int)
    r = np.arange(n)
    ms[r, r] = r + 1
    ms[0, n-1] = n + 1
    ms[n-1, 0] = n + 2
    return ms

In [42]:
ms = diagonal_plus_two(4)

dataset = MatrixDeterminantDataset(ms, with_multiplicity=True)
dataset

In [44]:
model = LeibnizModule(6, 10).cuda()
train_until_convergence(
    model,
    *loaders(dataset),
    regualizer_scaling_start=1000,
    verbose=True
)

[epoch 250] inter: 0.08171, extra: 0.07482
[epoch 500] inter: 0.003969, extra: 0.005359
[epoch 750] inter: 0.001085, extra: 0.001514
[epoch 1000] inter: 0.000476, extra: 0.0005767 | reg: 0.0701 (scale: 0.01)
[epoch 1250] inter: 0.000222, extra: 0.000239 | reg: 0.06955 (scale: 0.01284)
[epoch 1500] inter: 0.000115, extra: 0.0001169 | reg: 0.06855 (scale: 0.01648)
[epoch 1750] inter: 6.902e-05, extra: 6.582e-05 | reg: 0.06695 (scale: 0.02116)
[epoch 2000] inter: 4.608e-05, extra: 4.483e-05 | reg: 0.06436 (scale: 0.02717)
[epoch 2250] inter: 2.993e-05, extra: 3.102e-05 | reg: 0.06068 (scale: 0.03488)
[epoch 2500] inter: 2.14e-05, extra: 2.116e-05 | reg: 0.05605 (scale: 0.04478)
[epoch 2750] inter: 1.373e-05, extra: 1.206e-05 | reg: 0.05094 (scale: 0.0575)
[epoch 3000] inter: 4.175e-06, extra: 4.196e-06 | reg: 0.04519 (scale: 0.07382)
[epoch 3250] inter: 9.486e-07, extra: 1.112e-06 | reg: 0.0408 (scale: 0.09477)
[epoch 3500] inter: 2.108e-06, extra: 2.23e-06 | reg: 0.03627 (scale: 0.1217)


True

In [45]:
model.disp_equation(dataset.adapt_alphabet())

$-a_{5}a_{2}a_{3}a_{6}+a_{1}a_{2}a_{3}a_{4}$

In [115]:
n = 30
ms = diagonal_plus_two(n)

dataset = MatrixDeterminantDataset(ms, with_multiplicity=True)

inter_samplers = [
    one_mean_prod_sample
]
train_loader = dataset.dataloader(batch_size=64, samplers=[inter_sampler])
dataset_valid_interpolation_data = next(iter(dataset.dataloader(batch_size=10000, samplers=[inter_samplers[0]])))
dataset_test_extrapolation_data = next(iter(dataset.dataloader(batch_size=10000, samplers=[random_sign(one_mean_prod_sample)])))

# dataset

In [116]:
# _, _, t = dataset_valid_interpolation_data
# plt.hist(t.cpu().numpy().ravel())

In [120]:
model = LeibnizModule(n+2, 200).cuda()
train_until_convergence(
    model,
    train_loader,
    dataset_valid_interpolation_data,
    dataset_test_extrapolation_data,
    regualizer_scaling_start=3000,
    verbose=True,
    lr=1e-4
)

[epoch 250] inter: 3.294, extra: 1.107
[epoch 500] inter: 3.33, extra: 1.107
[epoch 750] inter: 3.445, extra: 1.107
[epoch 1000] inter: 3.419, extra: 1.107
[epoch 1250] inter: 3.571, extra: 1.107
[epoch 1500] inter: 3.728, extra: 1.107
[epoch 1750] inter: 3.736, extra: 1.107
[epoch 2000] inter: 3.67, extra: 1.107
[epoch 2250] inter: 3.627, extra: 1.107
[epoch 2500] inter: 3.682, extra: 1.107
[epoch 2750] inter: 3.599, extra: 1.107
[epoch 3000] inter: 3.698, extra: 1.107 | reg: 0.06088 (scale: 0.01)
[epoch 3250] inter: 1.151, extra: 1.107 | reg: 0.05177 (scale: 0.01284)
[epoch 3500] inter: 0.9988, extra: 1.107 | reg: 0.0462 (scale: 0.01648)
[epoch 3750] inter: 1.004, extra: 1.107 | reg: 0.0412 (scale: 0.02116)
[epoch 4000] inter: 1.04, extra: 1.107 | reg: 0.03673 (scale: 0.02717)
[epoch 4250] inter: 0.9863, extra: 1.107 | reg: 0.03271 (scale: 0.03488)
[epoch 4500] inter: 1.065, extra: 1.107 | reg: 0.02922 (scale: 0.04478)
[epoch 4750] inter: 1.061, extra: 1.107 | reg: 0.02631 (scale: 0.

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/infres/alacote/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-120-e4c6e0db3c56>", line 9, in <module>
    lr=1e-4
  File "<ipython-input-20-8f4153d14595>", line 40, in train_until_convergence
    loss_train_criterion = criterion(y_train, t_train) / var
  File "/home/infres/alacote/.local/lib/python3.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/infres/alacote/.local/lib/python3.7/site-packages/torch/nn/modules/loss.py", line 431, in forward
    return F.mse_loss(input, target, reduction=self.reduction)
  File "/home/infres/alacote/.local/lib/python3.7/site-packages/torch/nn/functional.py", line 2204, in mse_loss
    ret = torch._C._nn.mse_loss(expanded_input, expanded_target, _Reduction.get_enum(reduction))
KeyboardInterrupt

During han

KeyboardInterrupt: 

In [121]:
model.disp_equation(dataset.adapt_alphabet())

$$

In [47]:
from torch import nn

class ConvOperationBlock(nn.Module):
    
    def __init__(self, kernel_size=2, n_hidden=4, squared=True):
        super().__init__()
        self.kernel_size = kernel_size
        self.n_hidden = n_hidden
        self.conv = nn.Sequential(
            NMU(kernel_size**2, n_hidden, squared=squared),
            NAU(n_hidden, 1, squared=squared)
        )
        
    def forward(self, x):
        bs, h, w = x.size()
        lines = []
        for i in range(h-self.kernel_size+1):
            line = []
            for j in range(w-self.kernel_size+1):
                x_window = x[:, i:i+self.kernel_size, j:j+self.kernel_size]
                res = self.conv(x_window.reshape(bs, -1))
                line.append(res.squeeze())
            lines.append(torch.stack(line))
        return torch.stack(lines).permute(2, 0, 1)
    
    def __repr__(self):
        return f"COB({self.kernel_size}x{self.kernel_size}, n_hidden={self.n_hidden})"
    
class ConvOperationsBank(nn.Module):
    
    def __init__(self, n_out, kernel_size=2, n_hidden=4, squared=True, ravel=False):
        super().__init__()
        self.coblocks = []
        for i in range(n_out):
            coblock = ConvOperationBlock(kernel_size, n_hidden, squared)
            setattr(self, 'conv{}'.format(i), coblock)
            self.coblocks.append(coblock)
        self.ravel = ravel
        
    def forward(self, x):
        outs = []
        for coblock in self.coblocks:
            outs.append(coblock(x))
        out = torch.stack(outs).transpose(0, 1)
        if(self.ravel):
            return out.reshape(out.size(0), -1)
        else:
            return out

In [51]:
dataset = MatrixDeterminantDataset(ms, matrix_form=True)
model = nn.Sequential(
    ConvOperationsBank(5, kernel_size=2, n_hidden=5, ravel=True),
    NMU(5*9, 5),
    NAU(5, 1)
).cuda()
train_until_convergence(
    model,
    *loaders(dataset),
    regualizer_scaling_start=5000,
    verbose=True
)

[epoch 250] inter: 0.9917, extra: 1.109
[epoch 500] inter: 0.9917, extra: 1.109
[epoch 750] inter: 0.9917, extra: 1.109
[epoch 1000] inter: 0.9917, extra: 1.109


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/infres/alacote/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 2961, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-51-9a75d59119fd>", line 11, in <module>
    verbose=True
  File "<ipython-input-20-8f4153d14595>", line 25, in train_until_convergence
    y_train = model(x_train)
  File "/home/infres/alacote/.local/lib/python3.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "/home/infres/alacote/.local/lib/python3.7/site-packages/torch/nn/modules/container.py", line 92, in forward
    input = module(input)
  File "/home/infres/alacote/.local/lib/python3.7/site-packages/torch/nn/modules/module.py", line 541, in __call__
    result = self.forward(*input, **kwargs)
  File "<ipython-input-47-c7d0604e49e6>", line 43, in forward
    outs.append(coblock(x))
  File "/home/infres/alacote/.local/lib/python3.7

KeyboardInterrupt: 